### Алгоритм обучения нейронной сети класса CNN

- Подготовка и стандартизация данных;
- Моделирование, подбор параметров нейронной сети;
- Подбор параметров обучения;
- Обучение;
- Оценка качества обучения. 

### Описание кейса

Проведем обучение сверточной нейронной сети для задач распознавания рукописного текста

### Подготовка и стандартизация данных

In [ ]:
# Установим необходимые модули для обеспечения процесса:
import keras
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # библиотека базы выборок Mnist
from keras.utils import np_utils

# ﻿import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

### Подготовка и стандартизация данных

In [ ]:
# Установим необходимые модули для обеспечения процесса:
import keras
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # библиотека базы выборок Mnist
from keras.utils import np_utils

# ﻿import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
# За тренировочной и тестовой выборкой адресуем к библиотеке mnist 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# Проведем стандартизацию входных данных
x_train = x_train / 255
x_test = x_test / 255
# Преобразуем вектор класса (целые числа) в двоичную матрицу класса (нужно для categorical_crossenthropy)
y_train_cat = keras.utils.to_categorical(y_train, 10) 
y_test_cat = keras.utils.to_categorical(y_test, 10)

В библиотеке Keras сверточная нейронная сеть принимает тензор либо в формате ``` batch, channels, rows, cols``` или же в формате ``` batch, rows, cols, channels``` (размерность батчей, число строк, число столбцов у входного изображения или карты признаков, число каналов: если монохромное, то = 1, если цветное, то = 3, если карта признаков, то = числу фильтров). По умолчанию сейчас тензоры имеют формат ``` batch, rows, cols``` Функция ```np.expand_dims``` преобразует входные тензоры при помощи добавления оси размерности (0,1,2... n) Таким образом добавляя ось ```axis = 3``` мы добавляем в описание тензора недостающий ```...channels```

In [ ]:
# Преобразуем тензоры в читаемый нейронной сетью формат ``` batch, rows, cols, channels```
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

In [ ]:
# Проверим размерность обучающей выборки матрицы входных значений
print( x_train.shape ) 

In [ ]:
```60000``` количество изображений на вход
```28, 28, 1``` размерность изображения и количество цветовых каналов

### Моделирование, подбор параметров нейронной сети:

In [ ]:
# Установим необходимые модули для обеспечения процесса:
from tensorflow import keras
from tensorflow.keras.layers import Dense # Модуль описания слоев полносвязной нейронной сети (классификации признаков)
from tensorflow.keras.layers import Flatten # Модуль стандартизации (выравнивания) входных данных
from tensorflow.keras.layers import Conv2D # Модуль описания слоев свёрточной нейронной сети (извлечение признаков)
from tensorflow.keras.layers import MaxPooling2D # Модуль описания слоев субдискретизации (MaxPooling)
from tensorflow.keras.layers import Dropout # Модуль отсеивания (регуляризация)

In [ ]:
# Установим модуль для сохранения модели:
from tensorflow.keras.callbacks import ModelCheckpoint

Гипперпараметры ```Conv2D```:
- ```filters``` = число ядер импульксного отклика (каналов, фильтров)
- ```kernel_size``` = размер ядра
- ```padding``` = масштаб выходной карты признаков ``` same``` = такой же ```valid``` = с уменьшением
- ```input shape``` = формат фрагмента изображения в одной свёртке

Гипперпараметры ```MaxPooling2D```:
- ```Pool size``` = размер окна, в котором выбираются максимальные значения
- ```Strides``` = шаг сканирования по осям плоскости. По умолчанию = в 1 пиксел

In [ ]:
# Определим модель и гипперпараметры модели:
model = keras.Sequential([ # задаем модель для простого стека слоев (каркас с одним входным и выходным тензором)
    
    # Опишем модуль извлечения признаков (Cверточные C и слои субдискретизации S)
    Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)), # Параметры первого сверточного слоя (C1)
    MaxPooling2D((2, 2), strides=2), # Параметры первого слоя субдискретизации (S1)
    Conv2D(64, (3,3), padding='same', activation='relu'), # Параметры второго сверточного слоя (С2)
    MaxPooling2D((2, 2), strides=2), # Параметры второго слоя субдискретизации (S2)
    Flatten(), # преобразование тензора в вектор
    
    # Опишем модуль классификации и вывода (F слои полносвязной сети)
    Dense(128, activation='relu'), # Параметры скрытого слоя (F1)
    Dense(10,  activation='softmax') # Параметры выходного скрытого слоя (F1)
])

Поясним:
_Первая итерация извлечения признаков:
- ```Conv2D```(```32``` фильтра, размером ядра ```3x3```), масштаб выходного значения = ```такой же```, функция активация ядра = ```relu```, формат входного изображения ```28х28``` c ```1``` цветовым каналом
- ```MaxPooling2D```(размер окна ```2x2```), с шагом сканирования ```2``` пиксела (чтобы не пересекалась)
 
_Вторая итерация извлечения признаков:
- ```Conv2D```(```64``` фильтра...) остальное все идентично первой итерации. 

_После первой итерации размерность выходных значений уменьшилась вдвое и стала ```14x14x32```. После второй получаем тензор размерностью ```7x7``` и ```64``` каналла. ```Flatten``` преобразует этот тензор в вектор для последующей подачи на полносвязную нейронную сеть.  

_Классификация признаков:
- Скрытый слой задаем ```128``` нейронов,
- Выходной слой задаём ```10``` нейронов. (Почему ```softmax```???)

In [ ]:
# Выводим структуру модели в блокнот
print(model.summary())

Поясним:_
- Слой C1 ```conv2d```содержит ```320``` коэффициентов. Откуда эти цифры: ```32``` фильтра размером ```3x3 = 9``` пикселя + ```1``` bias = ```10```, ```32 x 10 = 320```

- Слой C2 ```conv2d_1```содержит ```18496``` коэффициентов. Откуда эти цифры: ```32``` фильтра размером ```3x3 = 9``` пикселя + ```1``` bias = ```10```, и умножаем еще на ```64``` фмльтра второго сверточного слоя```(3 x 3 x 32 + 1) * 64  = 18496```

- Слой F1 ```dense_2```содержит ```401536``` коэффициентов. Откуда эти цифры: подаваемый тензор на вход имеет формат ```7x7x64 + 1 = 3137``` пикселя, причем каждый из них связан с ```128``` коэффициентами изображения. Итого получаем ```3137 x 128 = 401536```

- Слой F1 ```dense_3```содержит ```1290``` коэффициентов. Откуда эти цифры: на выходной слой подаются ```128 + 1```  значений, которые связаны с ```10``` нейронами выходного слоя. Итого получаем ```129 * 10 = 1290```

### Подбор параметров обучения

В отличие от процессов обучения простых нейронных сетей, итерации обучения в Deep Learning занимают сравнительно продолжительное время. Происходит это из-за большого количества коэффициентов. Однако количество эпох сокращено на порядок. Задем параметры качества: инициализации входных значений весов, а также критерии качества 

In [ ]:
# Задаём параметры обучения
model.compile(optimizer='adam', # Метод оптимизации поиска начальных значений оставляем adam. в укачестве альтернативы: 'sgd'
             loss='categorical_crossentropy', # Функция потерь категориальная кросс-энтропия, т.к. у нас больше классов
             metrics=['accuracy']) # Метрика качества = точность, т.к. классификатор

In [ ]:
Определим критерии наилучшей модели, которая пойдёт в продакшен после обучения: 
```monitor``` = для отслеживания показателей проверки ```'val_loss'```
```verbose``` = показатель уровня логгирования модели ```'1'```
```save_best_only``` = сохраняем только лучшую модель ```'true'```

In [ ]:
# Определим, какие версии обученных моделей будем сохранять
filename = 'model.h15' # создадим checkpoint для сохранения модели в каждм цикле обучения
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

### Обучение нейронной сети

In [ ]:
Продолжительность обучения будет составлять ```5``` эпох:
Размер выборки валидации определим как ```20%```:
Изменение конфигурации весов будем проводить каждые ```32``` тензора: по количеству карт признаков на входной свертке

In [ ]:
# 1-2-3 Поехали! (старт обучение на обучающей выборке)
his = model.fit(x_train, y_train_cat, batch_size=32, epochs=5, validation_split=0.2)

In [ ]:
# Оцениваем качество обученной модели на тестовой выборке 
print('***** train')
model.evaluate(x_test, y_test_cat)